# spam or no spam

in this project we are gonna build a spam filter with naive bayes multinomial algorithm .


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
s=pd.read_csv("SMSSpamCollection",sep="\t",header=None,names=["label","sms"])

In [3]:
s.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
s.tail()

,label,sms
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [5]:
s.describe()

,label,sms
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [6]:
s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
label    5572 non-null object
sms      5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [7]:
s["label"].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: label, dtype: float64

from the above basic operations we can say that the dataset consists of 5572 entries all of which are not null. it has two columns namely label which tell us whether the message is spam or ham(non-spam).

86.59% are ham messages and 13.41% are spam messages.

In [8]:
randomized_dataset_s=s.sample(frac=1,random_state=1)
training_testing_index=round(len(randomized_dataset_s)*0.8)
train_set=randomized_dataset_s[:training_testing_index].reset_index(drop=True)
test_set=randomized_dataset_s[training_testing_index:].reset_index(drop=True)

In [9]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4458 entries, 0 to 4457
Data columns (total 2 columns):
label    4458 non-null object
sms      4458 non-null object
dtypes: object(2)
memory usage: 69.7+ KB


In [10]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1114 entries, 0 to 1113
Data columns (total 2 columns):
label    1114 non-null object
sms      1114 non-null object
dtypes: object(2)
memory usage: 17.5+ KB


In [11]:
train_set["label"].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: label, dtype: float64

In [12]:
test_set["label"].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: label, dtype: float64

Above we can clearly see that the train and the test dataset have same percentages for ham and spam. From this we can say that our sample is representative of the population.

In [13]:
train_set["sms"]=train_set["sms"].str.replace("\W"," ")
train_set["sms"]=train_set["sms"].str.lower()

In [14]:
train_set.head()

,label,sms
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [15]:
train_set["sms"]=train_set["sms"].str.split()
vocabulary=[]
for i in train_set["sms"]:
    for j in i:
        vocabulary.append(j)
vocabulary=list(set(vocabulary))

In [16]:
len(vocabulary)

7783

In [17]:
word_counts_per_sms={unique_word : [0]*len(train_set["sms"]) for unique_word in vocabulary}
for index,sms in enumerate(train_set["sms"]):
    for word in sms:
        word_counts_per_sms[word][index]+=1
        

In [18]:
word_count=pd.DataFrame(word_counts_per_sms)
word_count.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [19]:
train_set_clean=pd.concat([train_set,word_count],axis=1)
train_set_clean.head()

,label,sms,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [20]:
#spam and non spam messages
spam_messages=train_set_clean[train_set_clean["label"]=="spam"]
non_spam_messages=train_set_clean[train_set_clean["label"]=="ham"]
#p_spam and p_ham
p_spam=len(spam_messages)/len(train_set_clean)
p_ham=len(non_spam_messages)/len(train_set_clean)
#n_spam
n_words_per_message=spam_messages["sms"].apply(len)
n_spam=n_words_per_message.sum()
# n_ham
n_words_per_non_spam_message=non_spam_messages["sms"].apply(len)
n_ham=n_words_per_non_spam_message.sum()
# n_vocabulary
n_vocabulary=len(vocabulary)
print("p_spam:  ",p_spam)
print("p_ham:  ",p_ham)
print("n_spam:  ",n_spam)
print("n_ham:  ",n_ham)
print("n_vocabulary:  ",n_vocabulary)

p_spam:   0.13458950201884254
p_ham:   0.8654104979811574
n_spam:   1200
n_ham:   7716
n_vocabulary:   7783


In [21]:
alpha=1

### another way for n_spam and n_ham :

In [22]:
# first=spam_messages["sms"].iloc[0]
# print(first)

In [23]:
# len(first.iloc[0])

In [24]:
# first_ham=non_spam_messages["sms"].iloc[0]
# print(first_ham)
# print(len(first_ham.iloc[0]))

In [25]:
# def no_words(series):
#     n_words=0
#     for i in range(0,len(series)-1):
#         message_and_extra=series["sms"].iloc[i]
#         n_words+=len(first_ham.iloc[0])
#     return n_words

In [26]:
# n_spam_x=no_words(spam_messages)
# n_ham_x=no_words(non_spam_messages)
# print("n_spam: ",n_spam)
# print("n_ham: ",n_ham)

In [27]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = non_spam_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

TypeError: can only concatenate list (not "int") to list